In [83]:
import os
import sys
from itertools import *
import argparse
from collections import defaultdict

base = os.path.expanduser('~/Dropbox/School/Bonn/ProgrammingLab2')

# Exercise 1

## Exercise 1A

In [84]:
def read_fasta_sequence(file):
    header = next(file).strip()[1:]
    rest = "".join(line.strip() for line in file)
    return header, rest

with open(os.path.join(base, 'ecoli-genome.fna')) as f:
    hd, seq = read_fasta_sequence(f)
    print(hd)
    print(seq[:150])
    print(len(seq))

gi|556503834|ref|NC_000913.3| Escherichia coli str. K-12 substr. MG1655, complete genome
AGCTTTTCATTCTGACTGCAACGGGCAATATGTCTCTGTGTGGATTAAAAAAAGAGTGTCTGATAGCAGCTTCTGAACTGGTTACCTGCCGTGAGTAAATTAAAATTTTATTGACTTAGGTCACTAAATACTTTAACCAATATAGGCATA
4641652


## Exercise 1B

In [85]:
def read_fasta(file):
    for header, group in groupby(file, lambda line: line[0] == '>'):
        if header:
            h = next(group).strip()
        else:
            yield h, ''.join(l.strip() for l in group)

with open(os.path.join(base, 'ecoli-proteome.faa')) as f:
    l = list(read_fasta(f))
    
    mi = min(l, key=lambda header_seq: len(header_seq[1]))
    print("Minimum length sequence length: {}".format(len(mi[1])))
    
    ma = max(l, key=lambda header_seq: len(header_seq[1]))
    print("Maximum length sequence length: {}".format(len(ma[1])))
    

Minimum length sequence length: 14
Maximum length sequence length: 2358


## Exercise 1C

In [86]:
def write_fasta(outfile, header, sequence):
    print(">", header, sep='', file=outfile)
    chunks = [iter(sequence)] * 70
    for chunk in zip_longest(*chunks, fillvalue=''):
        print(''.join(chunk), file=outfile)
        
write_fasta(sys.stdout, 'header1', 'AGCTTTTCATTCTGACTGCAACGGGCAATATGTCTCTGTGTGGATTAAAAAAAGAGTGTCTGATAGCAGCTTCTGAAC')
write_fasta(sys.stdout, 'header2', 'AGCTTTTCATTCTGACTGCAACGGGCAATATGTCTCTGTGTGGATTAAAAAAAGAGTGTCTGATAGCAGCTTCTGAAC')

>header1
AGCTTTTCATTCTGACTGCAACGGGCAATATGTCTCTGTGTGGATTAAAAAAAGAGTGTCTGATAGCAGC
TTCTGAAC
>header2
AGCTTTTCATTCTGACTGCAACGGGCAATATGTCTCTGTGTGGATTAAAAAAAGAGTGTCTGATAGCAGC
TTCTGAAC


# Exercise 2

## Exercise 2A

In [87]:
transcribe_dict = {'A': 'T', 'T': 'A', 'C': 'G', 'G': 'C'}
transcribe = transcribe_dict.get

transcribe('A')

'T'

In [88]:
'ATCT'.translate(str.maketrans(transcribe_dict))

'TAGA'

In [89]:
def simulate_script_2a(argstring):
    parser = argparse.ArgumentParser()
    parser.add_argument("--input", "-i", default=sys.stdin, type=argparse.FileType('r'))
    parser.add_argument("--output", "-o", default=sys.stdout, type=argparse.FileType('w'))
    args = parser.parse_args(argstring.split())
    
    header, sequence = read_fasta_sequence(args.input)
    cdna_header = "cDNA_{}".format(header)
    cdna_sequence = reversed(list(map(transcribe, sequence)))
    write_fasta(args.output, cdna_header, cdna_sequence)

simulate_script_2a("--input {}".format(os.path.join(base, 'ecoli-genome-sample.fna')))

>cDNA_gi|556503834|ref|NC_000913.3| Escherichia coli str. K-12 substr. MG1655, complete genome
AAATTCCTGATCGACGAAAGTTTTCAATTGCGCCAGCGGGAACCCCGGCTGGGCGGCGGCGAGTCCCGTC
AAAAGTTCGGCAAAAATACGTTCGGCATCGCTGATATTGGGTAAAGCATCCTGGCCGCTAATGGTTTTTT
CAATCATCGCCACCAGGTGGTTGGTGATTTTGGCGGGGGCAGAGAGGACGGTGGCCACCTGCCCCTGCCT
GGCATTGCTTTCCAGAATATCGGCAACACGCAGAAAACGTTCTGCATTTGCCACTGATGTACCGCCGAAC
TTCAACACTCGCATGGTTGTTACCTCGTTACCTTTGGTCGAAAAAAAAAGCCCGCACTGTCAGGTGCGGG
CTTTTTTCTGTGTTTCCTGTACGCGTCAGCCCGCACCGTTACCTGTGGTAATGGTGATGGTGGTGGTAAT
GGTGGTGCTAATGCGTTTCATGGATGTTGTGTACTCTGTAATTTTTATCTGTCTGTGCGCTATGCCTATA
TTGGTTAAAGTATTTAGTGACCTAAGTCAATAAAATTTTAATTTACTCACGGCAGGTAACCAGTTCAGAA
GCTGCTATCAGACACTCTTTTTTTAATCCACACAGAGACATATTGCCCGTTGCAGTCAGAATGAAAAGCT


In [90]:
simulate_script_2a("-i {} -o {}".format(os.path.join(base, 'ecoli-genome.fna'),
                                        os.path.join(base, 'cth-ecoli-genome-complement.fna')))

## Exercise 2B


In [96]:
def find_frames(s, start_codons=set(['ATG']), stop_codons=set(['TAA', 'TAG', 'TGA'])):
    starts = defaultdict(list)
    stops = defaultdict(list)
    
    for i in range(len(s)):
        sl = s[i: i + 3]
        if sl in start_codons:
            starts[i % 3].append(i)
        elif sl in stop_codons:
            stops[i % 3].append(i)
    
    for i in range(3):
        if not starts[i]:
            continue
        
        orf_starts = starts[i]
        orf_stops = stops[i]
        orf_stops = [0] + orf_stops
        
        for j in range(1, len(orf_stops)):
            valid_starts = [start for start in orf_starts if orf_stops[j - 1] <= start < orf_stops[j]]
            if valid_starts:
                min_start = valid_starts[0]
                start_pos = min_start
                stop_pos = orf_stops[j] + 3
                yield (start_pos, stop_pos))

    #print('Starts at {}'.format(dict(starts)))
    #print('Stops at {}'.format(dict(stops)))
    #print('Valid Frames: {}'.format(frames))

test = "ATGATGxxATGxxATGATGxATGATGxxxxTAAxTAAxTAAxxxATGxATGxATG"
for start, stop in find_frames(test):
    print("{:03} {:03} {}".format(start, stop, test[start:stop]))

000 024 ATGATGxxATGxxATGATGxATGA
013 037 ATGATGxATGATGxxxxTAAxTAA
008 017 ATGxxATGA
020 041 ATGATGxxxxTAAxTAAxTAA


In [116]:
def find_longest_frames(seq):
    for start, stop in find_frames(seq):
        yield "gi|556503834|ref|NC_000913.3|:{}-{}".format(start + 1, stop), seq[start: stop]
        
    reverse_seq = seq[::-1].translate(str.maketrans(transcribe_dict))

    for start, stop in find_frames(reverse_seq):
        yield "gi|556503834|ref|NC_000913.3|:c{}-{}".format(len(seq) - start, 1 + len(seq) - stop), reverse_seq[start: stop]

        
test2 = test + test[::-1].translate(str.maketrans(transcribe_dict))
print("testing: {}, len: {}".format(test2, len(test2)))
list(find_longest_frames(test2))

testing: ATGATGxxATGxxATGATGxATGATGxxxxTAAxTAAxTAAxxxATGxATGxATGCATxCATxCATxxxTTAxTTAxTTAxxxxCATCATxCATCATxxCATxxCATCAT, len: 110


[('gi|556503834|ref|NC_000913.3|:1-24', 'ATGATGxxATGxxATGATGxATGA'),
 ('gi|556503834|ref|NC_000913.3|:14-37', 'ATGATGxATGATGxxxxTAAxTAA'),
 ('gi|556503834|ref|NC_000913.3|:9-17', 'ATGxxATGA'),
 ('gi|556503834|ref|NC_000913.3|:21-41', 'ATGATGxxxxTAAxTAAxTAA'),
 ('gi|556503834|ref|NC_000913.3|:c110-87', 'ATGATGxxATGxxATGATGxATGA'),
 ('gi|556503834|ref|NC_000913.3|:c97-74', 'ATGATGxATGATGxxxxTAAxTAA'),
 ('gi|556503834|ref|NC_000913.3|:c102-94', 'ATGxxATGA'),
 ('gi|556503834|ref|NC_000913.3|:c90-70', 'ATGATGxxxxTAAxTAAxTAA')]

In [117]:
def simulate_script_2b(argstring):
    parser = argparse.ArgumentParser()  
    parser.add_argument("--input", "-i", default=sys.stdin, type=argparse.FileType('r'))
    parser.add_argument("--output", "-o", default=sys.stdout, type=argparse.FileType('w'))
    args = parser.parse_args(argstring.split())
    
    for header, sequence in read_fasta(args.input):
        for label, frame in find_longest_frames(sequence):
            write_fasta(args.output, label, frame)
    
simulate_script_2b("--input {}".format(os.path.join(base, 'ecoli-genome-sample.fna')))

simulate_script_2b("-i{} -o {}".format(os.path.join(base, 'ecoli-genome-sample.fna'),
                                       os.path.join(base, 'cth-ecoli-orf-sample.ffn')))

>gi|556503834|ref|NC_000913.3|:190-255
ATGAAACGCATTAGCACCACCATTACCACCACCATCACCATTACCACAGGTAACGGTGCGGGCTGA
>gi|556503834|ref|NC_000913.3|:374-487
ATGCAGAACGTTTTCTGCGTGTTGCCGATATTCTGGAAAGCAATGCCAGGCAGGGGCAGGTGGCCACCGT
CCTCTCTGCCCCCGCCAAAATCACCAACCACCTGGTGGCGATGA
>gi|556503834|ref|NC_000913.3|:512-562
ATGCTTTACCCAATATCAGCGATGCCGAACGTATTTTTGCCGAACTTTTGA
>gi|556503834|ref|NC_000913.3|:30-98
ATGTCTCTGTGTGGATTAAAAAAAGAGTGTCTGATAGCAGCTTCTGAACTGGTTACCTGCCGTGAGTAA
>gi|556503834|ref|NC_000913.3|:c500-108
ATGGTTTTTTCAATCATCGCCACCAGGTGGTTGGTGATTTTGGCGGGGGCAGAGAGGACGGTGGCCACCT
GCCCCTGCCTGGCATTGCTTTCCAGAATATCGGCAACACGCAGAAAACGTTCTGCATTTGCCACTGATGT
ACCGCCGAACTTCAACACTCGCATGGTTGTTACCTCGTTACCTTTGGTCGAAAAAAAAAGCCCGCACTGT
CAGGTGCGGGCTTTTTTCTGTGTTTCCTGTACGCGTCAGCCCGCACCGTTACCTGTGGTAATGGTGATGG
TGGTGGTAATGGTGGTGCTAATGCGTTTCATGGATGTTGTGTACTCTGTAATTTTTATCTGTCTGTGCGC
TATGCCTATATTGGTTAAAGTATTTAGTGACCTAAGTCAATAA
>gi|556503834|ref|NC_000913.3|:c364-230
ATGTACCGCCGAACTTCAACACTCGCATGGTTGTTACCTCGTTACCTTTGGTCGAAAAAAAA

In [127]:
if not os.path.exists(os.path.join(base, 'cth-ecoli-orf.ffn')):
    simulate_script_2b("-i {} -o {}".format(os.path.join(base, 'ecoli-genome.fna'), 
                                            os.path.join(base, 'cth-ecoli-orf.ffn')))

# Exercise 3

## Exercise 3A

In [128]:
def get_sequence_positions(fasta_file):
    d = {}
    for header, sequence in read_fasta(fasta_file):
        annotations = header.split("|")
        start, stop = annotations[4].split()[0][1:].split('-')
        if start.startswith('c'):
            d[int(start[1:])] = int(stop)
        else:
            d[int(start)] = int(stop)
    return d

print("SAMPLE DATA")
with open(os.path.join(base, 'ecoli-orf-sample.ffn')) as f:
    d = get_sequence_positions(f) 
    for k in sorted(d):
        print(k, d[k])

print("\nCALCULATED")
with open(os.path.join(base, 'cth-ecoli-orf-sample.ffn')) as f:
    d = get_sequence_positions(f)
    for k in sorted(d):
        print(k, d[k])

SAMPLE DATA
30 98
187 170
190 255
364 230
374 487
500 108
512 562

CALCULATED
30 98
187 170
190 255
364 230
374 487
500 108
512 562


## Exercise 3B

In [134]:
def simulate_script_3b(argstring):
    parser = argparse.ArgumentParser()
    
    parser.add_argument("--genes", "-g", type=argparse.FileType('r'), required=True)
    parser.add_argument("--orfs", "-o", type=argparse.FileType('r'), required=True)
    args = parser.parse_args(argstring.split())
    
    genes = get_sequence_positions(args.genes)
    orfs = get_sequence_positions(args.orfs)
    
    correct = []
    for stop, start in orfs.items():
        if stop in genes and genes[stop] == start:
            correct.append(stop)
    
    n_orfs = len(orfs)
    n_genes = len(genes)
    n_correct = len(correct)
    correct_ratio = n_correct / n_genes
    n_correct_stops = len(set(orfs) & set(genes)) / n_orfs
    n_missed = n_genes - n_correct
    
    print("number orfs", n_orfs)
    print("number genes", n_genes)
    print("number correct", n_correct, "ratio", correct_ratio)
    print('number correct stops', n_correct_stops)
    print('number missed genes', n_missed)
    print('percent missed genes', n_missed / n_genes)
        
simulate_script_3b("-g {} -o {}".format(os.path.join(base, 'ecoli-genes.ffn'), 
                                        os.path.join(base, 'cth-ecoli-orf.ffn')))

number orfs 81687
number genes 4218
number correct 3138 ratio 0.7439544807965861
number correct stops 0.03869648781323834
number missed genes 1080
percent missed genes 0.25604551920341395


In [130]:
simulate_script_3b("-g {} -o {}".format(os.path.join(base, 'ecoli-genes.ffn'), 
                                        os.path.join(base, 'cth-ecoli-orf.ffn')))

number orfs 81687
number genes 4218
number correct 3138 ratio 0.038414925263505824
number correct stops 0.03869648781323834
number missed genes 1080
percent missed genes 0.25604551920341395


## Exercise 3C

In [135]:
def simulate_script_3c(argstring):
    parser = argparse.ArgumentParser()
    
    parser.add_argument("--genes", "-g", type=argparse.FileType('r'), required=True)
    parser.add_argument("--orfs", "-o", type=argparse.FileType('r'), required=True)
    parser.add_argument("--min-length", "-m", type=int, default=0)
    args = parser.parse_args(argstring.split())
    
    genes = get_sequence_positions(args.genes)
    orfs = get_sequence_positions(args.orfs)
    
    correct = []
    for stop, start in orfs.items():
        if stop in genes and genes[stop] == start:
            if stop > start and stop - start >= args.min_length:
                correct.append(stop)
            elif start - stop >= args.min_length:
                correct.append(stop)
        
    n_orfs = len(orfs)
    n_genes = len(genes)
    n_correct = len(correct)
    correct_ratio = n_correct / n_genes
    n_correct_stops = len(set(orfs) & set(genes)) / n_orfs
    n_missed = n_genes - n_correct
    
    print("minimum length", args.min_length)
    print("number orfs", n_orfs)
    print("number genes", n_genes)
    print("number correct", n_correct)
    print('number missed genes', n_missed)
    print("ratio correct", correct_ratio)
    print('number correct stops', n_correct_stops)
    
        

simulate_script_3c("-g {} -o {} -m 25".format(os.path.join(base, 'ecoli-genes.ffn'), 
                                              os.path.join(base, 'cth-ecoli-orf.ffn')))

minimum length 25
number orfs 81687
number genes 4218
number correct 3138
number missed genes 1080
ratio correct 0.7439544807965861
number correct stops 0.03869648781323834


# Exercise 4

In [132]:
def simulate_script_4(argstring):
    parser = argparse.ArgumentParser()
    
    parser.add_argument("--genes", "-g", type=argparse.FileType('r'), required=True)
    parser.add_argument("--orfs", "-o", type=argparse.FileType('r'), required=True)
    args = parser.parse_args(argstring.split())
    
    genes = get_sequence_positions(args.genes)
    orfs = get_sequence_positions(args.orfs)
    
    orf_labels = set(orfs)
    gene_labels = set(genes)
    
    # perform grid search over m's
    results = {}
    for m in range(0, 1200, 5):
        accepted = set(k for k, v in orfs.items() if k - v > m)
        rejected = orf_labels - accepted
        
        tp = len(accepted & gene_labels)
        fp = len(accepted) - tp
        
        fn = len(rejected & gene_labels)
        tn = len(rejected) - fn
    
        balanced_accuracy = 0.5 * (tp/(tp+fn) + tn/(tn+fp))
        results[m] = balanced_accuracy
        
    
    best_m = max(results, key=results.get)
    print('best cutoff was {}'.format(best_m))
    
simulate_script_4("-g {} -o {}".format(os.path.join(base, 'ecoli-genes.ffn'), 
                                       os.path.join(base, 'cth-ecoli-orf.ffn')))

best cutoff was 240
